In [ ]:
import csv
from pathlib import Path

import numpy as np

import torch
from torch import nn
import torch.nn.functional as tfunc
from torch.utils.data import DataLoader, Dataset

from matplotlib import pyplot as plt

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"using: {device}")

using: cuda


In [ ]:
def one_hot(n, i):
    q = np.zeros(n)
    q[i] = 1.0
    return q


class MNISTDataset(Dataset):
    def __init__(self, fp):

        xy = np.loadtxt(fp, delimiter=',', dtype=np.float32)

        self.x = torch.from_numpy(xy[:, 1:])
        self.x = self.x / 255.0

        labels = []
        for i in xy[:, 0]:
            labels.append(one_hot(10, int(i)))

        self.y = torch.from_numpy(np.array(labels))
        self.n_samples = xy.shape[0]

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

    def __len__(self):
        return self.n_samples

In [ ]:
train_dataset = MNISTDataset(Path('/content/sample_data/mnist_train_small.csv'))
test_dataset = MNISTDataset(Path('/content/sample_data/mnist_test.csv'))

In [ ]:
batch_size = 128

train_dataloader = DataLoader(train_dataset, batch_size)
test_dataloader = DataLoader(test_dataset, batch_size)

In [ ]:
class Perceptron(nn.Module):
    def __init__(self, n_in, n_hidden, n_out):
        super().__init__()

        self.n_in = n_in
        self.n_hidden = n_hidden
        self.n_out = n_out

        self.flatten = nn.Flatten()

        self.classifier = nn.Sequential(
            nn.Linear(self.n_in, self.n_hidden),
            nn.ReLU(),
            nn.Linear(self.n_hidden, self.n_out),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.flatten(x)
        y = self.classifier(x)
        return y

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    model.train()

    for (x, y) in dataloader:
        x, y = x.to(device), y.to(device)

        y_predicted = model(x)

        loss = loss_fn(y_predicted, y)
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()


In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    nbatches = len(dataloader)

    model.eval()

    loss = 0
    true_positive = 0

    with torch.no_grad():
        for (x, y) in dataloader:
            x, y = x.to(device), y.to(device)

            y_predicted = model(x)

            loss += loss_fn(y_predicted, y).item()
            true_positive += (y_predicted.argmax(1) == y.argmax(1)).type(torch.float).sum().item()

    return (loss / nbatches, true_positive / size)



In [ ]:
n_epoch = 16
learn_ing_rate = 1e-2

In [ ]:
n_in = 784
n_hidden = 16
n_out = 10

model = Perceptron(n_in, n_hidden, n_out).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
for i in range(n_epoch):
    train(train_dataloader, model, loss_fn, optimizer)
    loss, acc = test(test_dataloader, model, loss_fn)
    print(f"Epoch: {i+1}; loss: {round(loss, 3)}; accuracy: {round(acc * 100, 4)}%")

Epoch: 1; loss: 1.798; accuracy: 76.66%
Epoch: 2; loss: 1.672; accuracy: 83.04%
Epoch: 3; loss: 1.646; accuracy: 83.76%
Epoch: 4; loss: 1.634; accuracy: 84.3%
Epoch: 5; loss: 1.628; accuracy: 84.52%
Epoch: 6; loss: 1.624; accuracy: 84.68%
Epoch: 7; loss: 1.596; accuracy: 88.69%
Epoch: 8; loss: 1.569; accuracy: 90.73%
Epoch: 9; loss: 1.561; accuracy: 91.27%
Epoch: 10; loss: 1.556; accuracy: 91.74%
Epoch: 11; loss: 1.553; accuracy: 91.86%
Epoch: 12; loss: 1.551; accuracy: 92.01%
Epoch: 13; loss: 1.549; accuracy: 92.16%
Epoch: 14; loss: 1.547; accuracy: 92.22%
Epoch: 15; loss: 1.546; accuracy: 92.33%
Epoch: 16; loss: 1.545; accuracy: 92.45%
